In [15]:
import csv
import requests
import os
import sys
from yahoo_finance import Share
from collections import deque
from PIL import Image
import time
import finsymbols as symbols
from numpy.random import choice
import pandas as pd
from subprocess import call

In [2]:
## Text input

In [4]:
SP500dict = symbols.get_sp500_symbols()
SP500ticker = []
for i in SP500dict:
    SP500ticker.append(i['symbol'])
print((SP500ticker))
fd = open('labels.txt', 'w+')

[u'MMM', u'ABT', u'ABBV', u'ACN', u'ATVI', u'AYI', u'ADBE', u'AMD', u'AAP', u'AES', u'AET', u'AMG', u'AFL', u'A', u'APD', u'AKAM', u'ALK', u'ALB', u'ARE', u'ALXN', u'ALGN', u'ALLE', u'AGN', u'ADS', u'LNT', u'ALL', u'GOOGL', u'GOOG', u'MO', u'AMZN', u'AEE', u'AAL', u'AEP', u'AXP', u'AIG', u'AMT', u'AWK', u'AMP', u'ABC', u'AME', u'AMGN', u'APH', u'APC', u'ADI', u'ANDV', u'ANSS', u'ANTM', u'AON', u'AOS', u'APA', u'AIV', u'AAPL', u'AMAT', u'APTV', u'ADM', u'ARNC', u'AJG', u'AIZ', u'T', u'ADSK', u'ADP', u'AZO', u'AVB', u'AVY', u'BHGE', u'BLL', u'BAC', u'BK', u'BAX', u'BBT', u'BDX', u'BRK.B', u'BBY', u'BIIB', u'BLK', u'HRB', u'BA', u'BKNG', u'BWA', u'BXP', u'BSX', u'BHF', u'BMY', u'AVGO', u'BF.B', u'CHRW', u'CA', u'COG', u'CDNS', u'CPB', u'COF', u'CAH', u'CBOE', u'KMX', u'CCL', u'CAT', u'CBG', u'CBS', u'CELG', u'CNC', u'CNP', u'CTL', u'CERN', u'CF', u'SCHW', u'CHTR', u'CHK', u'CVX', u'CMG', u'CB', u'CHD', u'CI', u'XEC', u'CINF', u'CTAS', u'CSCO', u'C', u'CFG', u'CTXS', u'CLX', u'CME', u'CMS'

In [24]:
## Download Data

In [11]:
startMonth = "2"
startDay = "25"
startYear = "2010"
endMonth = "2"
endDay = "25"
endYear = "2018"
basePath = "/Users/patrick/Euler/S&P500/"
imagePath = "/Users/patrick/Euler/S&PImages/"
#basePathEdited = "/Users/tejassiripurapu/Desktop/"
#basePathSuperCharacter = "/Users/tejassiripurapu/Desktop/"

In [21]:
#for stockTicker in SP500ticker:
#print(stockTicker)
for stockTicker in SP500ticker:
    csvURL = "https://app.quotemedia.com/quotetools/getHistoryDownload.csv?&webmasterId=501&startDay=" + startDay + "&startMonth=" + startMonth + "&startYear=" + startYear + "&endDay=" + endDay + "&endMonth=" + endMonth + "&endYear=" + endYear + "&isRanged=true&symbol=" + stockTicker
    with requests.Session() as s:
        download = s.get(csvURL)
        fd = open(basePath + stockTicker + ".csv", 'wb')
        fd.write(download.content)
        fd.close()
        #break

In [22]:
csvURL = "https://app.quotemedia.com/quotetools/getHistoryDownload.csv?&webmasterId=501&startDay=" + startDay + "&startMonth=" + startMonth + "&startYear=" + startYear + "&endDay=" + endDay + "&endMonth=" + endMonth + "&endYear=" + endYear + "&isRanged=true&symbol=SPY"
with requests.Session() as s:
    download = s.get(csvURL)
    fd = open(basePath + "SPY" + ".csv", 'wb')
    fd.write(download.content)
    fd.close()

In [12]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# adjclose: 9, volume: 6, high: 2
# minimal spread between adjclose and high indicate up trend
def get_data_from_csv(fname):
    high_Price = []
    adj_Close = []
    volume = []
    
    if os.path.splitext(fname)[-1].lower() == '.csv':
        with open(fname,"rb") as csvFile:
            stockCSV = csv.reader(csvFile)
            stockCSV.next()
            for row in stockCSV:
                high_Price.append(row[1])
                adj_Close.append(row[8])
                volume.append(row[5])
    
    return high_Price, adj_Close, volume
    
def get_data_from_excel(fname):

    xl_workbook = xlrd.open_workbook(fname)
    xl_sheet = xl_workbook.sheet_by_index(0)

    open_price_sheet = xl_sheet.col(2)
    open_price = []
    for obj in open_price_sheet:
        open_price.append(obj.value)

    close_price_sheet = xl_sheet.col(5)
    close_price = []
    for obj in close_price_sheet:
        close_price.append(obj.value)

    vol_sheet = xl_sheet.col(6)
    vol = []
    for obj in vol_sheet:
        vol.append(obj.value)

    open_price.pop(0)
    close_price.pop(0)
    vol.pop(0)

    return open_price, close_price, vol

def scale_data(data_list):
    data_array = np.array(data_list)
    data_array = np.reshape(data_array, (-1,1))
    scaler = MinMaxScaler(feature_range=(0, 255))
    scaler.fit(data_array)
    scaled_data = scaler.transform(data_array)
    return scaled_data

def convert_to_images(open_data, close_data, vol_data):
    picture_list = [] 
    label_list = []
    percentage_growth = []


    count = 0
    
    for i in range(len(open_data)): # since this is for i in range it will produce a sliding window
        open_pic = open_data[i:i+224]
        close_pic = close_data[i:i+224]
        vol_pic = vol_data[i:i+224] 
        
        if len(open_pic) == 224:
            open_2D = [] 
            close_2D = [] 
            vol_2D = [] 

            for j in range(224): 
                open_2D.append(open_pic)
                close_2D.append(close_pic)
                vol_2D.append(vol_pic)

            open_2D = np.array(open_2D)
            close_2D = np.array(close_2D)
            vol_2D = np.array(vol_2D)
            final_pic = np.dstack((open_2D, close_2D, vol_2D))

            if len(open_data) > (i + 259):
                picture_list.append(final_pic)
                denom = float(open_data[i+224])
                if denom == 0:
                    denom = .001
                if open_data[i + 224] < open_data[i + 259]:
                    label_list.append(1)
                    percent = float(open_data[i+259] - open_data[i+224]) / denom
                    percentage_growth.append(percent)

                else:
                    label_list.append(0)
                    percent = float(open_data[i+259] - open_data[i+224]) / denom
                    percentage_growth.append(percent)

    return picture_list, label_list, percentage_growth

In [13]:
high_priceSP, close_priceSP, volSP = get_data_from_csv('/Users/patrick/Euler/S&P500/' + "SPY" + '.csv')

scaled_high = scale_data(high_priceSP)
scaled_high = np.flip(scaled_high, axis=0)

scaled_closed = scale_data(close_priceSP)
scaled_closed = np.flip(scaled_closed, axis=0)

scaled_vol = scale_data(volSP)
scaled_vol = np.flip(scaled_vol, axis=0)

pictures, labels, growthSP = convert_to_images(scaled_high, scaled_closed, scaled_vol)
completedTicker = []
skippedTickers = []
print(len(high_priceSP))

/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S9 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


2004


In [14]:
numProc = 0
startTime = time.time()
curTime = time.time()
#for i, stockTicker in enumerate(SP500ticker):
while(numProc < 505):
    stockTicker = SP500ticker.pop(0)
    numProc += 1
    high_price, close_price, vol = get_data_from_csv('/Users/patrick/Euler/S&P500/' + stockTicker + '.csv')
    if len(high_price) != len(high_priceSP) or len(close_price) != len(close_priceSP) or len(volSP) != len(vol):
        print("len not equal, stockTicker: " + stockTicker + str(len(high_price)))
        skippedTickers.append(stockTicker)
        continue
    #if(not(type(high_price) is float)):
    #    print("high not float, stockTicker: " + stockTicker)
    #    skippedTickers.append(stockTicker)        
    #    continue
    scaled_high = scale_data(high_price)
    scaled_high = np.flip(scaled_high, axis=0)
    #if(not(type(close_price) is float)):
    #    print("close not float")        
    #    skippedTickers.append(stockTicker)         
    #    continue
    scaled_closed = scale_data(close_price)
    scaled_closed = np.flip(scaled_closed, axis=0)
    #if(not(type(vol) is float)):
    #    print("vol not float")
    #    skippedTickers.append(stockTicker)        
    #    continue
    scaled_vol = scale_data(vol)
    scaled_vol = np.flip(scaled_vol, axis=0)

    pictures, labels, growth = convert_to_images(scaled_high, scaled_closed, scaled_vol)

    count = 0
    count0 = 0
    count1 = 0
    count2 = 0
    for picture in pictures:
        im = Image.fromarray(np.uint8(picture))
        im.save(imagePath + stockTicker + str(count) + '.png')
        difLabel = ""
        if growth[count] <= 0:
            difLabel = "0"
            count0 += 1
        elif growth[count] < growthSP[count]:
            difLabel = "1"
            count1 += 1
        else:
            difLabel = "2"
            count2 += 1
        fd.write(stockTicker + str(count) + '.png' + " " + difLabel + "\n")
        #print(stockTicker + str(count) + '.png' + " " + str(labels[count]) + "\n")
        #print("count: ", count)
        count += 1
        #if(count == 1):
        #break
    completedTicker.append(stockTicker)

    print("Time Remaining: " + str((time.time() - startTime) / numProc * 505 - (time.time() - startTime)) + " curTime: " + str(time.time() - curTime) + " count0: " + str(count0) + " count1: " + str(count1) + " count2: " + str(count2) + " numProcessed: " + str(numProc))
    curTime = time.time()
                                   
print("completed tickers")
print(completedTicker)
print("skipped tickers")
print(skippedTickers)


/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S7 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S7 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S9 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Time Remaining: 5135.09276104 curTime: 10.1886279583 count0: 560 count1: 237 count2: 948 numProcessed: 1
len not equal, stockTicker: AYI2003


/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S7 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S7 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S9 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Time Remaining: 3362.63227423 curTime: 9.90665698051 count0: 514 count1: 248 count2: 983 numProcessed: 3


/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S7 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S6 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S9 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


...

Time Remaining: 2542.65495747 curTime: 9.74151086807 count0: 786 count1: 220 count2: 739 numProcessed: 47


/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S9 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Time Remaining: 2577.36308959 curTime: 9.78017115593 count0: 701 count1: 273 count2: 771 numProcessed: 48


/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S7 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S7 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype |S9 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Time Remaining: 2612.83161461 curTime: 10.0570070744 count0: 734 count1: 137 count2: 874 numProcessed: 49
len not equal, stockTicker: APTV65
completed tickers
[u'ATVI', u'ADBE', u'AMD', u'AAP', u'AES', u'AET', u'AMG', u'AFL', u'A', u'AKAM', u'ALK', u'ALXN', u'ALGN', u'AGN', u'ADS', u'GOOGL', u'MO', u'AMZN', u'AAL', u'AIG', u'AMT', u'AMGN', u'ANDV', u'ANSS', u'ANTM', u'AIV', u'AAPL', u'AMAT']
skipped tickers
[u'AYI', u'APD', u'ALB', u'ARE', u'ALLE', u'LNT', u'ALL', u'GOOG', u'AEE', u'AEP', u'AXP', u'AWK', u'AMP', u'ABC', u'AME', u'APH', u'APC', u'ADI', u'AON', u'AOS', u'APA', u'APTV']


In [56]:
# equate label amounts for images
labels_df = pd.read_table("./labels.txt", sep=" ", names=["img", "label"])
print(labels_df.head())

zero = labels_df.loc[labels_df['label'] == 0.0]
one = labels_df.loc[labels_df['label'] == 1.0]
two = labels_df.loc[labels_df['label'] == 2.0]
zero = (zero.reset_index()).drop('index', axis=1)
one = (one.reset_index()).drop('index', axis=1)
two = (two.reset_index()).drop('index', axis=1)
print(zero.head())
print(one.head())
print(two.head())
min_len = min(len(zero), len(one), len(two))
zero_dellen = len(zero) - min_len
one_dellen = len(one) - min_len
two_dellen = len(two) - min_len
zero_todelete = choice(range(len(zero)), size=zero_dellen, replace=False)
one_todelete = choice(range(len(one)), size=one_dellen, replace=False)
two_todelete = choice(range(len(two)), size=two_dellen, replace=False)
print("minimum len: " + str(min_len))
print("zero orig len: " + str(len(zero)) + " len to delete: " + str((len(zero_todelete))))
print(zero_todelete)
print("one orig len: " + str(len(one)) + " len to delete: " + str((one_dellen)))
print(one_todelete)
print("two orig len: " + str(len(two)) + " len to delete: " + str((two_dellen)))
print(two_todelete)
zero_png = zero['img']
zero_lbls = (zero_png.ix[zero_todelete])
zero_lbls = list(zero_lbls)
#zero_lbls = (zero_lbls.reset_index()).drop('index', axis=1)
#print(zero_lbls.head())
print("zero new len: " + str(len(zero_lbls)))
one_png = one['img']
one_lbls = (one_png.ix[one_todelete])
one_lbls = list(one_lbls)
#one_lbls = (one_lbls.reset_index()).drop('index', axis=1)
two_png = two['img']
two_lbls = (two_png.ix[two_todelete])
two_lbls = list(two_lbls)
#two_lbls = (two_lbls.reset_index()).drop('index', axis=1)
#print(one_lbls.head())
print("one new len: " + str(len(one_lbls)))
#print(two_lbls.head())
print("two new len: " + str(len(two_lbls)))
count = 0
#print zero_lbls[0]
for i in zero_lbls:
    os.remove(imagePath + i)
    if(count < 5):
        print(i)
    count += 1
count = 0
for i in one_lbls:
    os.remove(imagePath + i)
    if(count < 5):
        print(i)
    count += 1
count = 0
for i in two_lbls:
    os.remove(imagePath + i)
    if(count < 5):
        print(i)
    count += 1

         img  label
0  ATVI0.png    2.0
1  ATVI1.png    2.0
2  ATVI2.png    2.0
3  ATVI3.png    2.0
4  ATVI4.png    2.0
          img  label
0  ATVI10.png    0.0
1  ATVI15.png    0.0
2  ATVI47.png    0.0
3  ATVI51.png    0.0
4  ATVI52.png    0.0
           img  label
0   ATVI77.png    1.0
1  ATVI154.png    1.0
2  ATVI155.png    1.0
3  ATVI156.png    1.0
4  ATVI159.png    1.0
         img  label
0  ATVI0.png    2.0
1  ATVI1.png    2.0
2  ATVI2.png    2.0
3  ATVI3.png    2.0
4  ATVI4.png    2.0
minimum len: 7507
zero orig len: 18308 len to delete: 10801
[15958  8852  5915 ...,  5762  1775  6543]
one orig len: 7507 len to delete: 0
[]
two orig len: 21898 len to delete: 14391
[ 5611  5944  9128 ..., 13754 15766 19611]
zero new len: 10801
one new len: 0
two new len: 14391
ANTM93.png
ANTM93.png
ALGN1471.png
A950.png
AAPL20.png
AMD1092.png
AFL159.png
AFL1005.png
ALXN590.png
GOOGL1009.png
ALXN124.png


In [ ]:
fd.close()

##Number of batches run, recorded manually (measured in number of companies)
# completed: 60 
# running currently: 50

In [58]:
import random

training = open("train.txt", "w+")
validation = open("val.txt", "w+")

with open('labels.txt') as f:
    for line in f:
        if random.uniform(0,1) < .7:
            training.write(line)
        else:
            validation.write(line)
            
training.close()
validation.close()